# Vektordatenbank Qdrant

1. Teil: **Quick Start**, siehe Qdrant-Doku: https://qdrant.tech/documentation/
2. Teil: Beispiel **Indexierung und Querying in Qdrant** &rarr; [Springe zu Teil 2](#beispiel-indexierung-und-querying-in-qdrant)


## Teil 1: Quick Start

#### 1. Qdrant Docker-Image herunterladen
`docker pull qdrant/qdrant`

#### 2. Container starten

`docker run -p 6333:6333 -p 6334:6334 \
    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
    qdrant/qdrant`


| Parameter                                    | Bedeutung        | 
| -------------------------------------------- | ---------------- | 
| `-p 6333:6333`                               | REST-API-Port    | 
| `-p 6334:6334`                               | gRPC-Port        |
| `-v $(pwd)/qdrant_storage:/qdrant/storage:z` | Lokaler Speicher &rarr; Daten bleiben erhalten |


Qdrant is now accessible:

    REST API: localhost:6333
    Web UI: localhost:6333/dashboard
    GRPC API: localhost:6334


#### 3. Client intitialisieren / verbinden

In [ ]:
# %pip install qdrant-client

In [3]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [4]:
print(client.get_collections())

collections=[]


#### 4. Neue Collection anlegen

In [ ]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.COSINE),
)

True

- importiert zwei Modelldefinitionen: 
    - `Distance` → legt fest, welche Ähnlichkeitsmetrik Qdrant benutzt, z. B. `COSINE`, `EUCLID`, `DOT`
    - `VectorParams` → beschreibt, wie die Vektoren aussehen (Dimension, Metrik, etc.)

- neue Collection `test_collection``
    - jeder Vektor hat 4 Dimensionen (`size=4`)
    - Adrant Skalarprodukt (DOT) als Distanzmetrik

##### Vectoren hinzufügen

In [8]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)


operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


#### Run a query

In [9]:
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

print(search_result)


[ScoredPoint(id=4, version=0, score=1.362, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=1.273, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=1.208, payload=None, vector=None, shard_key=None, order_value=None)]


In [22]:
# Suche mit index
search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.2, 0.1, 0.9, 0.7],
    limit=3,
    with_payload=True
)

/var/folders/2_/sb5q2w3x6zz5s2m0zwsdhy_c0000gn/T/ipykernel_73729/1812339302.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


#### Grundprinzip von Qdrant-Indexing

Qdrant kombiniert Vektor-Indizes (für schnelle Ähnlichkeitssuche) und Payload-Indizes (für Filter auf Metadaten).

Vector Index → beschleunigt die Suche nach ähnlichen Vektoren.

Payload Index → beschleunigt Filter auf Metadaten (z. B. „city = Berlin“).

Indexes werden pro Collection konfiguriert, nicht automatisch für jeden Segmentpunkt.

# Beispiel Indexierung und Querying in Qdrant

In [10]:
documents = [
    'Wie kann ich mein Passwort Passwort zurücksetzen?',
    'Wie ändere ich mein Passwort?',
    'Passwort vergessen - wie kann ich es zurücksetzen?',
    'Wie kann ich mein Login-Passwort erneuern?',
    'Wo finde ich meine Bestellhistorie?',
    'Wie kann ich meine bisherigen Bestellungen einsehen?',
    'Wo sehe ich meine vergangenen Bestellungen?',
    'Wo kann ich meine Bestellungen überprüfen?',
    'Bestellhistorie abrufen wie geht das?',
    'Wie kann ich meine Lieferadresse ändern?',
    'Lieferadresse aktualisieren wie geht das?',
    'Wie ändere ich meine Versandadresse?',
    'Kann ich meine Adresse nachträglich ändern?',
    'Adresse für Lieferung ändern möglich?',
    'Wie kontaktiere ich den Kundendienst?',
    'Wie erreiche ich den Support?',
    'Kundendienst kontaktieren wie?',
    'Wo kann ich den Kundenservice erreichen?',
    'Wie bekomme ich Hilfe vom Support-Team?',
    'Welche Zahlungsmethoden werden akzeptiert?',
    'Welche Bezahlmöglichkeiten gibt es?',
    'Wie kann ich bezahlen?',
    'Akzeptierte Zahlungsmethoden - Übersicht',
    'Welche Zahlungsarten stehen zur Verfügung?',
    'Wie kann ich meine Bestellung stornieren?',
    'Bestellung rückgängig machen wie?',
    'Wie annulliere ich eine Bestellung?',
    'Kann ich meine Bestellung noch stornieren?',
    'Stornierung einer Bestellung Anleitung',
    'Wie lange dauert der Versand?',
    'Versanddauer - wie lange dauert es?',
    'Wann wird meine Bestellung geliefert?',
    'Lieferzeitraum - wie lang ist er?',
    'Wie schnell kommt meine Bestellung an?',
    'Kann ich Artikel nach der Bestellung noch ändern?',
    'Kann ich meine Bestellung nachträglich bearbeiten?',
    'Artikel in einer bereits getätigten Bestellung ändern - geht das?',
    'Bestellung nachträglich ändern - möglich?',
    'Kann ich Produkte nach der Bestellung austauschen?'
]

print(f'{len(documents)} documents loaded.')

39 documents loaded.


#### Vektoren generieren

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

vectors = model.encode(documents)
print(f'{len(vectors)} vectors created, each of length {len(vectors[0])}')


39 vectors created, each of length 384


#### Collection erstellen (Vektorindex)

In [ ]:

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# Verbindung zu lokalem Qdrant
client = QdrantClient(host="localhost", port=6333)

# Collection anlegen mit Vektorindex
client.recreate_collection(
    collection_name="faq_collection",
    vectors_config=VectorParams(size=len(vectors[0]), 
    distance=Distance.COSINE),
)


/var/folders/2_/sb5q2w3x6zz5s2m0zwsdhy_c0000gn/T/ipykernel_73729/1914378725.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

- `vectors_config=VectorParams(size=..., distance=...)` teilt Qdrant mit, wie groß die Vektoren sind und welche Distanzfunktion für die Suche verwendet werden soll. 
- `size` = Länge der Embeddings, gibt die Dimension der Vektoren an, also wie viele einzelne Werte jeder Vektor hat. `len(vectors[0])` ist die Länge des ersten Vektors aus der Liste, also etwa 128 oder 384 je nach Modell.
- `distance` = Cosine Similarity.

#### Payload-Index erstellen (optional)

In [24]:
from qdrant_client.models import KeywordIndexParams

# Payload-Feld 'category' als Keyword für Filter markieren
client.create_payload_index(
    collection_name="faq_collection",
    field_name="category",
    field_schema=KeywordIndexParams(type="keyword")
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

- Payloadindex: beschleunigt Filter, z. B. nur Dokumente einer Kategorie.
- Hier Keyword → exakte Übereinstimmungen möglich.

#### Dokumente indexieren (`upsert`) | in Qdrant speichern
- Einfügen der Vektoren als „Points“ zusammen mit Metadaten (Payload) in die Collection.

In [26]:
from qdrant_client.models import PointStruct

points = [
    PointStruct(
        id=i, 
        vector=vectors[i], 
        payload={"text": documents[i], "category": "faq"})
    for i in range(len(documents))
]

client.upsert(
    collection_name="faq_collection",
    points=points,
    wait=True
)

print("All documents indexed in Qdrant")

All documents indexed in Qdrant


#### Query: Ähnliche Dokumente suchen

In [42]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

query = "Passwort vergessen"

# Query Text in Vektor umwandeln
query_vector = model.encode([query])[0] # 1D-Array (384,)

search_results = client.query_points(
    collection_name="faq_collection",
    query=query_vector,
    query_filter=Filter(must=[FieldCondition(key="category", match=MatchValue(value="faq"))]),
    with_payload=True,
    limit=5
).points

print("Top 5:")
for hit in search_results:    
    print(f"Score: {hit.score:.3f} | Doc {hit.id}: {hit.payload['text']}")


Top 5:
Score: 0.794 | Doc 2: Passwort vergessen - wie kann ich es zurücksetzen?
Score: 0.660 | Doc 0: Wie kann ich mein Passwort Passwort zurücksetzen?
Score: 0.618 | Doc 1: Wie ändere ich mein Passwort?
Score: 0.500 | Doc 3: Wie kann ich mein Login-Passwort erneuern?
Score: 0.356 | Doc 30: Versanddauer - wie lange dauert es?


- Jedes Dokument bekommt einen Embedding-Vektor.
- `PointStruct` speichert den Vektor + Metadaten (payload).
- `upsert` fügt alle Punkte in Qdrant ein (Indexierung).
- `search` findet die ähnlichsten Dokumente zu einer Query.